In [51]:
import numpy as np
import pandas as pd
import re
import csv
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_similarity_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import regularizers, initializers, optimizers, callbacks

In [75]:
labels = ["anger", "disgust", "fear", "happy", "sad", "surprise", "neutral"]
data = pd.read_csv("tweets.csv")

In [76]:
# remove the unnamed column
data = data[["Tweet", "clean tweets", "emotion"]]
data.to_csv("finaltweet.csv")

In [77]:
texts, labels, clean = [], [], []
with open('finaltweet.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        texts.append(row[1])
        labels.append(row[3])
        clean.append(row[2])

In [78]:
labels = labels[1:]
clean = clean[1:]

In [116]:
clean

['maxkellerman also helps majority nfl coaching inept bill obriens play calling wow gopats',
 'roommate okay cant spell autocorrect terrible firstworldprobs',
 'rooneys fucking untouchable isnt fucking dreadful depay looked decentishtonight',
 'bossupjaee pussy weak heard stfu bitch got threaten pregnant',
 'girl hit carnot get lucky w scratch also spared wrath sleep deprived kait🙃',
 'btuk broadband shocking regretting signing angry shouldofgonewithvirgin',
 'people need look definition protest protesting called vandalism angry stop',
 'bitchesthecat look teef growl',
 'star trek online update download oh fuming yay',
 'cant stop finished dejected luckily one bathroom go stall wait pants dry',
 'nhlexpepicks usahockey usa embarrassing watch last time guys game horrible joke',
 'glee filled normy dry humping recent high profile celebrity break pathetic amp wrong world today',
 'zorefx guys see next video fucking horrid',
 'fucking muppet jrwild1 stalker',
 'cat bloody lucky rspca weren

In [80]:
MAX_NB_WORDS = 40000
MAX_SEQUENCE_LENGTH = 30
VALIDATION_SPLIT = 0.2

In [81]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(clean)
sequences = tokenizer.texts_to_sequences(clean)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [117]:
data

array([[    0,     0,     0, ...,   723,   233,  3487],
       [    0,     0,     0, ...,  3488,    62,  6609],
       [    0,     0,     0, ...,  9279,   847,  9280],
       ...,
       [    0,     0,     0, ...,   376,   959,    14],
       [    0,     0,     0, ...,     8,   398,   475],
       [    0,     0,     0, ...,     0,     0, 29178]], dtype=int32)

In [82]:
from sklearn.model_selection import train_test_split

In [83]:
from keras.utils.np_utils import to_categorical
labels = to_categorical(np.asarray(labels))
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [84]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(data, labels , test_size=0.3)

In [42]:
EMBEDDING_DIM = 100
GLOVE_DIR = "glove.twitter.27B.100d.txt"
embeddings_index = {}
f = open(GLOVE_DIR)
print("Loading GloVe from:",GLOVE_DIR,"...",end="")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print("Done.\nProceeding with Embedding Matrix...")
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print("Completed!")

Loading GloVe from: glove.twitter.27B.100d.txt ...Done.
Proceeding with Embedding Matrix...
Completed!


In [64]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [65]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32') # 1000
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(embedded_sequences)
l_cov2= Conv1D(128, 3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_cov1)
l_pool1 = MaxPooling1D(3)(l_cov2)
#l_drop1 = Dropout(0.2)(l_pool1)
l_cov3 = Conv1D(128, 3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_pool1)
l_pool4 = MaxPooling1D(6)(l_cov3) # global max pooling
l_flat = Flatten()(l_pool4)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(7, activation='softmax')(l_dense) # 4 categories


In [67]:
def step_cyclic(epoch):
    try:
        if epoch%11==0:
            return float(2)
        elif epoch%3==0:
            return float(1.1)
        elif epoch%7==0:
            return float(0.6)
        else:
            return float(1.0)
    except:
        return float(1.0)
        
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=256, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoints", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(step_cyclic)

In [113]:
import pandas as pd
tweets = [
    "Watching the sopranos again from start to finish!",
    "Finding out i have to go to the  dentist tomorrow",
    "I want to go outside and chalk but I have no chalk",
    "I HATE PAPERS AH #AH #HATE",
    "My mom wasn't mad",
    "Do people have no Respect for themselves or you know others peoples homes",
]

def convert(texts):
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    word_index = tokenizer.word_index
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    return data


In [86]:
k = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=0.01, rho=0.95, epsilon=None, decay=0.01)
k.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])
k.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 30)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 30, 100)           2917900   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 28, 128)           38528     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 26, 128)           49280     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 8, 128)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 6, 128)            49280     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 1, 128)            0         
__________

In [87]:
k.fit(Xtrain, Ytrain,
          epochs=200, batch_size=128,
          callbacks=[tensorboard, model_checkpoints, lr_schedule])

Epoch 1/200
13910/13910 [==============================] - 5s 366us/step - loss: 1.2963 - acc: 0.5701
Epoch 2/200
  384/13910 [..............................] - ETA: 4s - loss: 1.2067 - acc: 0.5729

/Users/aviralsharma/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


13910/13910 [==============================] - 5s 351us/step - loss: 1.1905 - acc: 0.5988
Epoch 3/200
13910/13910 [==============================] - 4s 322us/step - loss: 1.1619 - acc: 0.6094
Epoch 4/200
13910/13910 [==============================] - 5s 325us/step - loss: 1.1475 - acc: 0.6142
Epoch 5/200
13910/13910 [==============================] - 4s 320us/step - loss: 1.1343 - acc: 0.6221
Epoch 6/200
13910/13910 [==============================] - 5s 341us/step - loss: 1.1251 - acc: 0.6228
Epoch 7/200
13910/13910 [==============================] - 5s 328us/step - loss: 1.1193 - acc: 0.6235
Epoch 8/200
13910/13910 [==============================] - 4s 320us/step - loss: 1.1093 - acc: 0.6270
Epoch 9/200
13910/13910 [==============================] - 5s 368us/step - loss: 1.1091 - acc: 0.6279
Epoch 10/200
13910/13910 [==============================] - 6s 410us/step - loss: 1.1055 - acc: 0.6285
Epoch 11/200
13910/13910 [==============================] - 6s 465us/step - loss: 1.1016 - ac

In [127]:
y_hat = k.predict(Xtest)
count = 0
for i in range(len(y_hat)):
    if np.argmax(y_hat[i]) == np.argmax(Ytest[i]):
        count += 1

print(100 * count / len(y_hat))

55.93760483059376


In [95]:
saved_model = k.to_json()
with open("twitter.json", "w") as json_file:
    json_file.write(saved_model)
model.save_weights("twitter.h5")

In [120]:
y_hat = k.predict(tweets)
for i in range(len(y_hat)):
    print(np.argmax(y_hat[i])) 

6
0
0
6
6
0
